# Clean AI Tools with Dependency Injection

## ActionContext

In [ ]:
########################################################################
# The ActionContext serves as a container for any resources that tools
# might need during their execution.
########################################################################
class ActionContext:
    def __init__(self, properties: Dict=None):
        self.context_id = str(uuid.uuid4())
        self.properties = properties or {}

    def get(self, key: str, default=None):
        return self.properties.get(key, default)

    def get_memory(self):
        return self.properties.get("memory", None)

In [ ]:
########################################################################
# Now we can modify our tool to accept an ActionContext, allowing it
# to access the conversation history without knowing where it comes
# from:
########################################################################
@register_tool(
    description="Analyze code quality and suggest improvements",
    tags=["code_quality"]
)
def analyze_code_quality(action_context: ActionContext, code: str) -> str:
    """Review code quality and suggest improvements."""
    # Get memory to understand the code's context
    memory = action_context.get_memory()
    
    # Extract relevant history
    development_context = []
    for mem in memory.get_memories():
        if mem["type"] == "user":
            development_context.append(f"User: {mem['content']}")
        # Hypotethical scenario where our agent includes the phrase "Here's the implementation" when it generates code
        elif mem["type"] == "assistant" and "Here's the implementation" in mem["content"]:
            development_context.append(f"Implementation Decision: {mem['content']}")
    
    # Create review prompt with full context
    review_prompt = f"""Review this code in the context of its development history:

Development History:
{'\n'.join(development_context)}

Current Implementation:
{code}

Analyze:
1. Does the implementation meet all stated requirements?
2. Are all constraints and considerations from the discussion addressed?
3. Have any requirements or constraints been overlooked?
4. What improvements could make the code better while staying within the discussed parameters?
"""
    
    generate_response = action_context.get("llm")
    return generate_response(review_prompt)

In [ ]:
########################################################################
# This pattern of dependency injection through ActionContext becomes
# even more valuable when we need to handle authentication-specific
# information. Consider a tool that needs to update a project
# management system with the status of code reviews. This requires
# authentication, but we don’t want to hardcode credentials or
# configure them at the tool level.
#
# Here’s how we can use ActionContext to handle authentication:
########################################################################
@register_tool(
    description="Update code review status in project management system",
    tags=["project_management"]
)
def update_review_status(action_context: ActionContext, 
                        review_id: str, 
                        status: str) -> dict:
    """Update the status of a code review in the project system."""
    # Get the authentication token for this specific request
    auth_token = action_context.get("auth_token")
    if not auth_token:
        raise ValueError("Authentication token not found in context")
    
    # Make authenticated request
    headers = {
        "Authorization": f"Bearer {auth_token}",
        "Content-Type": "application/json"
    }
    
    response = requests.post(
        f"https://...someapi.../reviews/{review_id}/status",
        headers=headers,
        json={"status": status}
    )
    
    if response.status_code != 200:
        raise ValueError(f"Failed to update review status: {response.text}")
        
    return {"status": "updated", "review_id": review_id}

# When the agent runs a tool, it provides all necessary context through
# the ActionContext:

def run(self, user_input: str, memory=None, action_context_props=None):
    """Execute the agent loop."""
    memory = memory or Memory()
    
    # Create context with all necessary resources
    action_context = ActionContext({
        'memory': memory,
        'llm': self.generate_response,
        # Request-specific auth
        **action_context_props
    })
    
    while True:
        prompt = self.construct_prompt(action_context, self.goals, memory)
        response = self.prompt_llm_for_action(action_context, prompt)
        result = self.handle_agent_response(action_context, response)
        
        if self.should_terminate(action_context, response):
            break

########################################################################
# Run the agent and create custom context for the action to 
# pass to tools that need it
########################################################################
some_agent.run("Update the project status...", 
               memory=..., 
               # Pass request-specific auth token
               action_context_props={"auth_token": "my_auth_token"})
    

## The Environment

In [ ]:
########################################################################
# Now that we have our ActionContext to hold shared resources and
# dependencies, we face a new challenge: how do we get it to just the
# tools that need it? We need a solution that provides dependencies
# selectively, giving each tool access to only the resources it
# requires.
# Consider that many tools are simple and self-contained, needing only
# their explicit parameters. A basic string manipulation tool
# shouldn't receive memory access or authentication tokens it doesn't
# use. Not only would this add unnecessary complexity, but it could
# also create security concerns by exposing sensitive information to
# tools that don’t need it.
# We can implement this logic in our environment system, which can
# examine each tool’s requirements and provide only the dependencies
# it specifically requests. Consider how much cleaner the agent’s code
# becomes.
#
# The environment examines each tool’s function signature and
# automatically injects the dependencies it needs. This happens
# through two mechanisms:
#
# 1. Special parameter names like action_context are automatically
# injected
#
# 2. Properties from the action_context can be accessed by prefixing
# the parameter name with _
########################################################################
def handle_agent_response(self, action_context: ActionContext, response: str) -> dict:
    """Handle action without dependency management."""
    action_def, action = self.get_action(response)
    result = self.environment.execute_action(self, action_context, action_def, action["args"])
    return result

class PythonEnvironment(Environment):
    def execute_action(self, agent, action_context: ActionContext, 
                      action: Action, args: dict) -> dict:
        """Execute an action with automatic dependency injection."""
        try:
            # Create a copy of args to avoid modifying the original
            args_copy = args.copy()

            # If the function wants action_context, provide it
            if has_named_parameter(action.function, "action_context"):
                args_copy["action_context"] = action_context

            # Inject properties from action_context that match _prefixed parameters
            for key, value in action_context.properties.items():
                param_name = "_" + key
                if has_named_parameter(action.function, param_name):
                    args_copy[param_name] = value

            # Execute the function with injected dependencies
            result = action.execute(**args_copy)
            return self.format_result(result)
        except Exception as e:
            return {
                "tool_executed": False,
                "error": str(e)
            }

In [ ]:
########################################################################
# To hide the dependencies from the agent, we need to update our tool
# registration system to ignore these special parameters when building
# the schema that the agent uses.
#
# This system gives us a clean separation of concerns:
# 
# 1. The agent focuses on deciding what actions to take
# 2. Tools declare what dependencies they need
# 3. The environment handles dependency injection and result management
# 4. ActionContext provides a flexible container for shared resources
#
########################################################################
def get_tool_metadata(func, tool_name=None, description=None, 
                     parameters_override=None, terminal=False, 
                     tags=None):
    """Extract metadata while ignoring special parameters."""
    signature = inspect.signature(func)
    type_hints = get_type_hints(func)

    args_schema = {
        "type": "object",
        "properties": {},
        "required": []
    }

    for param_name, param in signature.parameters.items():
        # Skip special parameters - agent doesn't need to know about these
        if param_name in ["action_context", "action_agent"] or \
           param_name.startswith("_"):
            continue

        # Add regular parameters to the schema
        param_type = type_hints.get(param_name, str)
        args_schema["properties"][param_name] = {
            "type": "string"  # Simplified for example
        }

        if param.default == param.empty:
            args_schema["required"].append(param_name)

    return {
        "name": tool_name or func.__name__,
        "description": description or func.__doc__,
        "parameters": args_schema,
        "tags": tags or [],
        "terminal": terminal,
        "function": func
    }